In [1]:
import requests
from bs4 import BeautifulSoup
import re
from razdel import tokenize
import pandas as pd

special_regex = re.compile('\d{2}.\d{2}.\d{2}')
emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)
with open('../DATA/less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()


main_url = 'https://leader-id.ru/'
user_suburl = 'users/'
test_user_id = '2234799' 

headers_for_users = [
    'Интересы',
    'Активность',
    # 'Проекты',
    'Команды'
]

In [2]:
def textNormalization(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', ' ', text)
    text = ' '.join(list(filter(lambda a: len(a), text.split(' '))))
    return text

def clear_from_stopwords(text, sw):
    text = text
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        if len(i.text) > 2:
            if i.text not in sw:
                t.append(i.text)

    return ' '.join(t)


In [3]:
ms = requests.get(main_url + user_suburl + test_user_id).content
soup = BeautifulSoup(ms, 'html.parser').find(class_='page')

In [4]:
def processing(block):
    if textNormalization(block.h3.text) == 'Интересы':
        return set(filter(len, [clear_from_stopwords(textNormalization(i.text), stop_words).lower() for i in  block.find_all('span')]))
    if textNormalization(block.h3.text) == 'Команды':
        return {clear_from_stopwords(textNormalization(j.text), sw=stop_words).lower(): clear_from_stopwords(textNormalization(i.text), sw=stop_words).lower() for i, j in zip(block.find_all('h4'), block.find_all('p'))}
    if textNormalization(block.h3.text) == 'Активность':
        return {clear_from_stopwords(textNormalization(i.text), sw=stop_words).lower(): i['href'] for i in block.find_all('a')}
        # return {textNormalization(j.text): textNormalization(i.text) for i, j in zip(block.find_all('h4'), block.find_all('p'))}
    return block

In [5]:
for i in set([i for i in soup.find_all('h3') if textNormalization(i.text) in headers_for_users]):
    print(processing(i.find_parents()[2]))

{'торжественное открытие ит-хаба окружного хакатона цифров ...': '/events/468237', 'защита учебной практики студентами кафедры прикладная ...': '/events/440288', 'итоговая защита проектов пои 2023': '/events/421421', 'хакатон разработке it-решения партнер омскстат': '/events/423140', '122-й математический it-субботник': '/events/376962', 'введение nlp': '/events/402031', 'экспертиза проектов выполняемых рамках выпускных квали ...': '/events/407353', 'частые практики дизайн-мышлению': '/events/402028'}
{'большие данные', 'квантовые технологии', 'образование', 'edunet', 'energynet', '2234799', 'профессиональное', 'игропрактики', 'управление проектами', 'мероприятие технологической команды', 'птк', 'журналистика', 'беспроводная связь вещей', 'машинное обучение анализ данных', 'foodnet', 'autonet', 'gamenet', 'дополнительное', 'личное профессиональное развитие', 'technet', 'искусственный интеллект машинное обучение', 'михаил александрович смирнов', 'программирование', 'neuronet', 'wearnet'